In [11]:
# Calculating Young's modulus from Abaqus
force = 200
a = 5E-3
Area = a**2
Stress = force/Area
Max_strain = 0.1298
E = Stress/Max_strain
print("{:e}Pa".format(E))
print(str(E/1E9)+' GPa')

6.163328e+07Pa
0.06163328197226502 GPa


In [12]:
#IPL struts
import math
import numpy as np

def bcc_strut_density(a, h, t):
    """
    Calculate the density of a BCC unit cell with strut thickness t, accounting for overlap.

    :param a: lattice parameter, or the edge length of the BCC unitK cell.
    :param t: thickness of the struts.
    :return: density of the unit cell struts.
    """
    # Each space diagonal can be calculated as sqrt(3) * a.
    length_strut = math.sqrt(a**2+a**2+h**2)

    # Calculate the volume of one strut.
    volume_strut = math.pi * (t / 2)**2 * length_strut  # volume = pi * r^2 * h

    # Calculate the overlapped volume at the center of the unit cell.
    overlap_volume = math.pi * (t / 2)**2 * t

    # Calculate the total volume of struts in the unit cell minus the overlapped volume.
    total_volume_struts = 4 * volume_strut - overlap_volume

    # Calculate the volume of the unit cell.
    volume_unit_cell = a**3

    # Calculate the density of the struts in the unit cell.
    density = total_volume_struts / volume_unit_cell

    return density

In [13]:
# Getting strut thickness from relative density required
import math

def bcc_strut_thickness(a, relative_density):
    """
    Calculate the strut thickness given relative density of a BCC unit cell.

    :param a: lattice parameter, or the edge length of the BCC unit cell.
    :param relative_density: relative density of the unit cell struts.
    :return: strut thickness.
    """

    # Calculate the length of one strut (space diagonal).
    length_strut = math.sqrt(3) * a

    # Calculate the volume of the unit cell.
    volume_unit_cell = a ** 3

    # Calculate the total volume of struts in the unit cell.
    total_volume_struts = relative_density * volume_unit_cell

    # Solve for t
    t = 2 * math.sqrt((total_volume_struts) / (4 * math.pi * length_strut))

    return t

# Example usage:
a = 2.5  # Suppose the lattice parameter a is 1
relative_density = 0.034825  # Suppose the relative density is 0.1
t = bcc_strut_thickness(a, relative_density)
t


0.20000014991173504

In [14]:
# FCC struts
import math

def fcc_strut_density(a, t):
    """
    Calculate the density of an FCC unit cell with strut thickness t.

    :param a: lattice parameter, or the edge length of the FCC unit cell.
    :param t: thickness of the struts.
    :return: density of the unit cell struts.
    """

    # The FCC unit cell contains 12 equal edge struts.
    # Each strut can be considered as a cylinder.

    # Calculate the length of one strut (which is equal to the lattice parameter).
    length_strut = a

    # Calculate the volume of one strut.
    volume_strut = math.pi * (t / 2)**2 * length_strut  # volume = pi * r^2 * h

    # Calculate the total volume of struts in the unit cell.
    total_volume_struts = 12 * volume_strut

    # Calculate the volume of the unit cell.
    volume_unit_cell = a**3

    # Calculate the density of the struts in the unit cell.
    density = total_volume_struts / volume_unit_cell

    return density

# Example usage:
a = 1  # Suppose the lattice parameter a is 1
t = 0.1  # Suppose the strut thickness t is 0.1
density = fcc_strut_density(a, t)
print(f"Density of the FCC unit cell struts: {density:.6f}")

Density of the FCC unit cell struts: 0.094248


In [15]:
# Stiffness from Paper (Theoretical and numerical analysis of the mechanical responses of BCC and FCC lattice structures)
import math
def stiffness_equation(a, b, d, r, Eo):
  stiffness = 4*a*Eo/(b*d*((a**2)/(math.pi*r**2*math.sqrt(a**2+b**2+d**2))+((b**2+d**2)*math.sqrt(a**2+b**2+d**2)/(12*math.pi*r**4))))
  return stiffness

In [16]:
# Calculating density, modulus, speed of sound
import numpy as np
import pandas as pd

# unit_cell_size = [1.42857, 1.66666, 2.0, 2.5, 3.333, 4, 5]
# strut_radius = [0.075, 0.1, 0.125, 0.145, 0.175]

unit_cell_size = [1.42857, 1.66666, 2.0, 2.5]
strut_radius = [0.125]

Eo = 190E9  # Example value for Eo
rho_316 =  7980
n=2.69

# The columns will be 'Strut Thickness (2x) mm'
index = ['{:.3g} mm'.format(2*r) for r in strut_radius]
# Formatting the index to include 'mm' for unit cell size
columns = ['{:.1f} mm'.format(a) for a in unit_cell_size]

# Initialize an empty DataFrame with specified columns and index
df = pd.DataFrame(index=index, columns=columns)
density_df = pd.DataFrame(index=index, columns=columns)
E_df = pd.DataFrame(index=index, columns=columns)
speed_sound_df = pd.DataFrame(index=index, columns=columns)

# Fill the DataFrame with stiffness values
for a in unit_cell_size:
    for r in strut_radius:
        E_beam_bending_1 = stiffness_equation(a, a, a, r, Eo)
        E_beam_bending_2 = 0.5*stiffness_equation(a, a/math.sqrt(2), a/math.sqrt(2), r, Eo)
        E_beam_bending = E_beam_bending_1 + E_beam_bending_2 
        # E_beam_bending = E_beam_bending_2
        # Accessing cells by the formatted string of the unit cell size and strut thickness
        df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(E_beam_bending/1E9)

        density_1 = bcc_strut_density(a, a, 2*r)
        density_2 = 0.5*bcc_strut_density(a/math.sqrt(2), a, 2*r)
        density = density_1 + density_2
        print(density_1, density_2)
        density_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density)
        E_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density**2*Eo/1E9)
        speed_sound_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(0.5*math.sqrt(Eo*density**n/(density*rho_316)))


# Apply formatting to avoid standard scientific notation
df = df.applymap(lambda x: '{:.3f}'.format(float(x)))
density_df = density_df.applymap(lambda x: '{:.3f}'.format(float(x)))
E_df = E_df.applymap(lambda x: '{:.3f}'.format(float(x)))
speed_sound_df = speed_sound_df.applymap(lambda x: '{:.2f}'.format(float(x)))

print('Relative Density df (Strut thickness (row), Unit Cell Size (column))')
print(density_df)
print(' ')
print('E effective (GPa) from Cellular Solids')
print(E_df)
print(' ')
print('E effective (GPa) from Beam Bending')
print(df)
print(' ')
print('Sound speed df')
print(speed_sound_df)

0.16243389396543859 0.18647014808303655
0.11978168593299666 0.1376240729280611
0.08348786441059332 0.09600539399003342
0.05362858276362909 0.06172113233725629
Relative Density df (Strut thickness (row), Unit Cell Size (column))
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm  0.349  0.257  0.179  0.115
 
E effective (GPa) from Cellular Solids
         1.4 mm  1.7 mm 2.0 mm 2.5 mm
0.25 mm  23.129  12.589  6.121  2.528
 
E effective (GPa) from Beam Bending
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm  1.613  0.879  0.427  0.176
 
Sound speed df
          1.4 mm  1.7 mm  2.0 mm  2.5 mm
0.25 mm  1002.15  775.03  571.50  393.32


In [17]:
# Modulus varying with strut thickness and unit cell size

# Fill the DataFrame with stiffness values

m_list = np.linspace(2.65, 2.8, 16)
for m in m_list:
  for a in unit_cell_size:
      for r in strut_radius:
        E_beam_bending_1 = stiffness_equation(a, a, a, r, Eo)
        E_beam_bending_2 = stiffness_equation(a, a/math.sqrt(2), a/math.sqrt(2), r, Eo)
        E_beam_bending = E_beam_bending_1 + E_beam_bending_2 
        # Accessing cells by the formatted string of the unit cell size and strut thickness
        df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(E_beam_bending/1E9)

        density_1 = bcc_strut_density(a, a, 2*r)
        density_2 = 0.5*bcc_strut_density(a/math.sqrt(2), a, 2*r)
        density = density_1 + density_2
        density_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density)
        E_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density**3*Eo/1E9)
        speed_sound_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(math.sqrt(Eo*density**n/(density*rho_316)))

  E_df = E_df.applymap(lambda x: '{:.2f}'.format(float(x)))
  print('E effective from Cellular Solids (GPa) and n:', m)
  print(E_df)
  print('')

E effective from Cellular Solids (GPa) and n: 2.65
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm   8.07   3.24   1.10   0.29

E effective from Cellular Solids (GPa) and n: 2.6599999999999997
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm   8.07   3.24   1.10   0.29

E effective from Cellular Solids (GPa) and n: 2.67
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm   8.07   3.24   1.10   0.29

E effective from Cellular Solids (GPa) and n: 2.6799999999999997
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm   8.07   3.24   1.10   0.29

E effective from Cellular Solids (GPa) and n: 2.69
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm   8.07   3.24   1.10   0.29

E effective from Cellular Solids (GPa) and n: 2.6999999999999997
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm   8.07   3.24   1.10   0.29

E effective from Cellular Solids (GPa) and n: 2.71
        1.4 mm 1.7 mm 2.0 mm 2.5 mm
0.25 mm   8.07   3.24   1.10   0.29

E effective from Cellular Solids (GPa) and n: 2.7199999999999998
        1.4 mm 1.7 mm 2.0

In [32]:
# Modulus varying with strut thickness and unit cell size

# Fill the DataFrame with stiffness values

# unit_cell_size = [1.0, 1.1111, 1.25, 1.42857, 1.66666, 2.0, 2.5]
# strut_radius = [0.075, 0.1, 0.125, 0.145, 0.175]

unit_cell_size = [2, 2.5, 3.0, 4, 5]
strut_radius = [0.075, 0.1, 0.125, 0.145, 0.175]



m=2.69

Eo = 190E9  # Example value for Eo
rho_316 =  7980
n=2.69

# The columns will be 'Strut Thickness (2x) mm'
index = ['{:.3g} mm'.format(2*r) for r in strut_radius]
# Formatting the index to include 'mm' for unit cell size
columns = ['{:.1f} mm'.format(a) for a in unit_cell_size]

# Initialize an empty DataFrame with specified columns and index
df = pd.DataFrame(index=index, columns=columns)
density_df = pd.DataFrame(index=index, columns=columns)
E_df = pd.DataFrame(index=index, columns=columns)
speed_sound_df = pd.DataFrame(index=index, columns=columns)

for a in unit_cell_size:
    for r in strut_radius:
      E_beam_bending_1 = stiffness_equation(a, a, a, r, Eo)
      E_beam_bending_2 = stiffness_equation(a, a/math.sqrt(2), a/math.sqrt(2), r, Eo)
      E_beam_bending = E_beam_bending_1 + E_beam_bending_2 
      # Accessing cells by the formatted string of the unit cell size and strut thickness
      df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(E_beam_bending/1E9)

      density_1 = bcc_strut_density(a, a, 2*r)
      density_2 = 0.5*bcc_strut_density(a/math.sqrt(2), a, 2*r)
      density = density_1 + density_2
      density_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density)
      E_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density**2*Eo/1E9)
      speed_sound_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(math.sqrt(Eo*density**n/(density*rho_316)))

E_df = E_df.applymap(lambda x: '{:.3f}'.format(float(x)))
print('E effective from Cellular Solids (GPa) and n:', m)
print(E_df)
print('')

df = df.applymap(lambda x: '{:.3f}'.format(float(x)))
print('Relative Density')
print(df)
print('')

E effective from Cellular Solids (GPa) and n: 2.69
         2.0 mm 2.5 mm 3.0 mm 4.0 mm 5.0 mm
0.15 mm   0.806  0.332  0.161  0.051  0.021
0.2 mm    2.528  1.042  0.505  0.161  0.066
0.25 mm   6.121  2.528  1.226  0.391  0.161
0.29 mm  11.011  4.553  2.210  0.705  0.290
0.35 mm  23.129  9.584  4.658  1.488  0.613

Relative Density
        2.0 mm 2.5 mm 3.0 mm 4.0 mm 5.0 mm
0.15 mm  0.096  0.039  0.019  0.006  0.002
0.2 mm   0.301  0.124  0.060  0.019  0.008
0.25 mm  0.729  0.301  0.146  0.046  0.019
0.29 mm  1.311  0.542  0.263  0.084  0.034
0.35 mm  2.748  1.142  0.555  0.177  0.073



In [18]:
import numpy as np
import pandas as pd

unit_cell_size = [1.0, 1.1111, 1.25, 1.42857, 1.66666, 2.0, 2.5]
strut_radius = [0.075, 0.1, 0.125, 0.145, 0.175]
Eo = 210  # Example value for Eo
rho_316 =  7.98

# The columns will be 'Strut Thickness (2x) mm'
index = ['{:.3g} mm'.format(2*r) for r in strut_radius]
# Formatting the index to include 'mm' for unit cell size
columns = ['{:.1f} mm'.format(a) for a in unit_cell_size]

# Initialize an empty DataFrame with specified columns and index
df = pd.DataFrame(index=index, columns=columns)
density_df = pd.DataFrame(index=index, columns=columns)
E_df = pd.DataFrame(index=index, columns=columns)
speed_sound_df = pd.DataFrame(index=index, columns=columns)

# Fill the DataFrame with stiffness values
for a in unit_cell_size:
    for r in strut_radius:
        stiffness = stiffness_equation(a, a, a, r, Eo)
        # Accessing cells by the formatted string of the unit cell size and strut thickness
        df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(2000*stiffness)

        density = bcc_strut_density(a, 2*r)
        density_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density)
        E_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(density*Eo)
        speed_sound_df.loc['{:.3g} mm'.format(2*r), '{:.1f} mm'.format(a)] = '{:.3f}'.format(math.sqrt(density**2*Eo/(density*rho_316)))


# Apply formatting to avoid standard scientific notation
df = df.applymap(lambda x: '{:.2f}'.format(float(x)))
density_df = density_df.applymap(lambda x: '{:.2f}'.format(float(x)))
E_df = E_df.applymap(lambda x: '{:.2f}'.format(float(x)))
speed_sound_df = speed_sound_df.applymap(lambda x: '{:.2f}'.format(float(x)))

print('Stiffness df (N/mm)')
print(df)
print(' ')
print('Relative Density df (Strut thickness (row), Unit Cell Size (column))')
print(density_df)
print(' ')
print('E effective df (GPa)')
print(E_df)
print(' ')
print('Sound speed df (GPa)')
print(speed_sound_df)

TypeError: bcc_strut_density() missing 1 required positional argument: 't'